In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import re
import nltk
nltk.download('punkt')
from tqdm.auto import tqdm
from wordcloud import WordCloud
import html
from langdetect import detect, detect_langs, LangDetectException
from google_trans_new import google_translator
import random
from collections import Counter
import unicodedata
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jeremyrava/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Roadmap to Text Mining

Text mining est le processus d'extraction d'informations à partir de grandes quantités de données non structurées. Cela implique plusieurs étapes, qui peuvent être décomposées en ces étapes suivantes :

1. Collecte des données : La première étape du text mining est la collecte de grande quantité de données non structurées. Cette donnée peut provenir de diverses sources telles que les plateformes sociaux, les sites Web, les blogs et les articles de presse.

2. Prétraitement : Une fois les données recueillies, elles doivent être prétraitées. Cela consiste à nettoyer les données en supprimant les caractères indésirables ou les mots, en corrigeant les erreurs orthographiques et en convertissant le texte en minuscule.

3. Tokenisation : Une fois les données prétraitées, elles sont tokenisées. Cela signifie que le texte est divisé en mots individuels ou des tokens. Cette étape est importante car elle permet d'analyser la fréquence de chaque mot dans le texte.

4. Suppression des mots-clés : Une fois le texte tokenisé, nous pouvons supprimer les mots-clés. Les mots-clés sont des mots courants tels que "le", "et" et "est" qui n'apportent pas de sens particulier et peuvent être supprimés sans affecter l'analyse globale.

5. Reduction morphologique ou normalisation du lemme : La prochaine étape est la reduction morphologique ou la normalisation du lemme. Cela consiste à réduire chaque mot à sa forme de base, également connu sous le nom de racine de mot. Par exemple, "courir" serait réduit à "court". Cela est fait pour regrouper des mots similaires ensemble, comme différents temps verbaux du même mot.

6. Extraction des fonctionnalités : Une fois le texte prétraité et nettoyé, nous pouvons extraire les fonctionnalités à partir de lui. Ces fonctionnalités peuvent inclure des choses telles que la fréquence des mots, l'analyse du sentiment ou le modélisation des sujets.

7. Analyse : Enfin, nous pouvons analyser les données à l'aide de diverses techniques telles que l'agrégation ou la classification pour obtenir des informations sur le texte.

In [2]:
# CONSTANTES
FIGSIZE = (12, 4)
PATH_DATA = "../data/"
PATH_RAW = PATH_DATA + "raw/"
PATH_PROCESSED = PATH_DATA + "processed/"
PATH_EXTERNAL = PATH_DATA + "external/"

pd.set_option("display.max_columns", 20)
pd.set_option("display.max_rows", 200)

In [12]:
X_train = pd.read_csv(PATH_RAW + "x_train.csv", index_col=0)
X_test = pd.read_csv(PATH_RAW + "x_test.csv", index_col=0)
display(X_train.head())
display(X_test.head())

target = pd.read_csv(PATH_RAW + "y_train.csv", index_col=0)

display(target.describe())
df_origin = pd.read_csv(PATH_RAW + "x_train.csv", index_col=0)

,designation,description,productid,imageid
0,Olivia: Personalisiertes Notizbuch / 150 Seite...,NaN,3804725264,1263597046
1,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...,NaN,436067568,1008141237
2,Grand Stylet Ergonomique Bleu Gamepad Nintendo...,PILOT STYLE Touch Pen de marque Speedlink est ...,201115110,938777978
3,Peluche Donald - Europe - Disneyland 2000 (Mar...,NaN,50418756,457047496
4,La Guerre Des Tuques,Luc a des id&eacute;es de grandeur. Il veut or...,278535884,1077757786


,designation,description,productid,imageid
84916,Folkmanis Puppets - 2732 - Marionnette Et Théâ...,NaN,516376098,1019294171
84917,Porte Flamme Gaxix - Flamebringer Gaxix - 136/...,NaN,133389013,1274228667
84918,Pompe de filtration Speck Badu 95,NaN,4128438366,1295960357
84919,Robot de piscine électrique,<p>Ce robot de piscine d&#39;un design innovan...,3929899732,1265224052
84920,Hsm Destructeur Securio C16 Coupe Crois¿E: 4 X...,NaN,152993898,940543690


,prdtypecode
count,84916.000000
mean,1773.219900
std,788.179885
min,10.000000
25%,1281.000000
50%,1920.000000
75%,2522.000000
max,2905.000000


In [13]:
stop_words_french = pd.read_json(PATH_EXTERNAL + "stop_words_french.json")
print(stop_words_french.shape)
stop_words = []
stop_words.extend(stop_words_french[0].tolist())
stop_words.extend(["cm", "mm"])
print(len(stop_words))
stop_words[:10]

(496, 1)
498


['a', 'à', 'â', 'abord', 'afin', 'ah', 'ai', 'aie', 'ainsi', 'allaient']

# PRE-PROCESSING LEXIQUE FRANÇAIS

In [14]:
def strip_accents(texts):
    sentences = []
    for sentence in tqdm(texts):
        if sentence is np.nan:
            sentences.append(np.nan)
        else:
            s = "".join(
                c
                for c in unicodedata.normalize("NFD", sentence)
                if unicodedata.category(c) != "Mn"
            )
            sentences.append(s)
    return sentences

In [15]:
lexique = pd.read_table(PATH_EXTERNAL + "Lexique383/Lexique383.tsv", delimiter="\t")
lexique_filtred_unique = lexique.drop_duplicates(subset="ortho")[["ortho", "lemme"]]
lexique_filtred_unique.dropna(subset=["ortho", "lemme"], axis="index", inplace=True)
# lexique_filtred_unique["ortho"] = strip_accents(lexique_filtred_unique["ortho"])
# lexique_filtred_unique["lemme"] = strip_accents(lexique_filtred_unique["lemme"])
lexique_filtred_unique = lexique_filtred_unique.drop_duplicates(subset="ortho")[
    ["ortho", "lemme"]
]

lexique_filtred_unique

,ortho,lemme
0,a,a
3,a capella,a capella
4,a cappella,a cappella
5,a contrario,a contrario
6,a fortiori,a fortiori
...,...,...
142685,ôtèrent,ôter
142686,ôté,ôter
142688,ôtée,ôter
142690,ôtées,ôter


# Premier nettoyage
### pour pouvoir traduire les textes d'une autre langue en français

In [16]:
def first_cleaning(sentences):
    # Pre-compile regex patterns
    HTML_TAGS_RE = re.compile("<[^>]*")
    URL_RE = re.compile(r"https?://[-_.?&~;+=/#0-9A-Za-z]{1,2076}")
    MAIL_RE = re.compile(
        r"[-_.0-9A-Za-z]{1,64}@[-_0-9A-Za-z]{1,255}[-_.0-9A-Za-z]{1,255}"
    )
    SPE_CHAR_RE = re.compile("[^a-zA-ZÀ-ÿ]")
    SPACES_RE = re.compile("\s+")
    SPACES_RE_2 = re.compile(" +")

    sentences_cleaned = []

    for sentence in tqdm(sentences):
        # Decode HTML entities
        sentence = html.unescape(sentence)

        # Replace HTML tags with spaces
        sentence = HTML_TAGS_RE.sub("", sentence)

        # Remove URL and e-mail adress
        sentence = URL_RE.sub("", sentence)
        sentence = MAIL_RE.sub("", sentence)

        # Replace special characters with spaces while keeping accents
        sentence = SPE_CHAR_RE.sub(" ", sentence)

        # Remove unnecessary spaces from the sentence using regular expressions
        cleaned_sentence = SPACES_RE.sub(" ", sentence)
        cleaned_sentence = SPACES_RE_2.sub(" ", cleaned_sentence)
        sentences_cleaned.append(cleaned_sentence)

    return sentences_cleaned

X_train

In [ ]:
X_train["designation"] = first_cleaning(X_train["designation"])
X_train["description"].loc[~X_train["description"].isna()] = first_cleaning(
    X_train["description"].loc[~X_train["description"].isna()]
)

# join texts
X_train["text"] = np.where(
    X_train["description"].isna(),
    X_train["designation"].astype(str),
    X_train["designation"].astype(str) + " " + X_train["description"].astype(str),
)
X_train.head()

X_test

In [17]:
X_test["designation"] = first_cleaning(X_test["designation"])
X_test["description"].loc[~X_test["description"].isna()] = first_cleaning(
    X_test["description"].loc[~X_test["description"].isna()]
)

# join texts
X_test["text"] = np.where(
    X_test["description"].isna(),
    X_test["designation"].astype(str),
    X_test["designation"].astype(str) + " " + X_test["description"].astype(str),
)
X_test.head()

  0%|          | 0/13812 [00:00<?, ?it/s]

  0%|          | 0/8926 [00:00<?, ?it/s]

/var/folders/_z/rfghqg0j31d_t9y5_yrj2ss00000gn/T/ipykernel_52227/4077928039.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test["description"].loc[~X_test["description"].isna()] = first_cleaning(


,designation,description,productid,imageid,text
84916,Folkmanis Puppets Marionnette Et Théâtre Mini ...,NaN,516376098,1019294171,Folkmanis Puppets Marionnette Et Théâtre Mini ...
84917,Porte Flamme Gaxix Flamebringer Gaxix U Twilig...,NaN,133389013,1274228667,Porte Flamme Gaxix Flamebringer Gaxix U Twilig...
84918,Pompe de filtration Speck Badu,NaN,4128438366,1295960357,Pompe de filtration Speck Badu
84919,Robot de piscine électrique,Ce robot de piscine d un design innovant et é...,3929899732,1265224052,Robot de piscine électrique Ce robot de pisci...
84920,Hsm Destructeur Securio C Coupe Crois E X Mm,NaN,152993898,940543690,Hsm Destructeur Securio C Coupe Crois E X Mm


# Detection des langues

In [18]:
def detect_language(sentences):

    UNKNOWN = "unknown"
    FR = "fr"

    languages = []
    k = 15
    lexique_set = set(lexique_filtred_unique["ortho"].str.strip().str.lower())

    for sentence in tqdm(sentences):
        # Split the sentence into words
        words = nltk.word_tokenize(sentence, language="french")

        # If there are more than k words, sample k words randomly.
        if len(words) > k:
            words = random.sample(words, k)

        # Use a list comprehension with a ternary conditional operator for language detection.
        df_la = []
        for word in words:
            if word.lower() in lexique_set:
                df_la.append(FR)
            else:
                try:
                    df_la.append(detect(word))
                except LangDetectException:
                    df_la.append(UNKNOWN)

        # Use Counter to find the most common language.
        reel_lang = Counter(df_la).most_common(1)[0][0] if len(df_la) > 0 else UNKNOWN
        languages.append(reel_lang)

    return languages

X_train

In [ ]:
X_train["language"] = detect_language(X_train["text"])
X_train.to_csv(PATH_PROCESSED + "X_train_detected_language.csv")
display(X_train.head())
X_train["language"].value_counts()

X_test

In [22]:
X_test["language"] = detect_language(X_test["text"])
X_test.to_csv(PATH_PROCESSED + "X_test_detected_language.csv")
display(X_test.head())
X_test["language"].value_counts()

  0%|          | 0/13812 [00:00<?, ?it/s]

,designation,description,productid,imageid,text,language
84916,Folkmanis Puppets Marionnette Et Théâtre Mini ...,NaN,516376098,1019294171,Folkmanis Puppets Marionnette Et Théâtre Mini ...,fr
84917,Porte Flamme Gaxix Flamebringer Gaxix U Twilig...,NaN,133389013,1274228667,Porte Flamme Gaxix Flamebringer Gaxix U Twilig...,fr
84918,Pompe de filtration Speck Badu,NaN,4128438366,1295960357,Pompe de filtration Speck Badu,fr
84919,Robot de piscine électrique,Ce robot de piscine d un design innovant et é...,3929899732,1265224052,Robot de piscine électrique Ce robot de pisci...,fr
84920,Hsm Destructeur Securio C Coupe Crois E X Mm,NaN,152993898,940543690,Hsm Destructeur Securio C Coupe Crois E X Mm,fr


language
fr    13124
en      295
de      113
tl       29
it       28
ro       25
fi       21
so       20
sl       19
id       17
pt       13
pl       12
no       11
es       11
sw       10
af        9
nl        9
ca        8
cs        7
cy        7
da        6
lt        6
sv        3
tr        3
sq        2
hu        1
et        1
vi        1
sk        1
Name: count, dtype: int64

# Traduction en français

In [23]:
def translate_texts(translator: google_translator, sentences, language):
    texts_translated = translator.translate(sentences, lang_tgt="fr", lang_src=language)
    return texts_translated

X_train

In [ ]:
X_train = pd.read_csv(PATH_PROCESSED + "X_train_detected_language.csv", index_col=0)

# si error durant la traduction -> reprendre à l'index done
done = 16

# google translate API
translator = google_translator(timeout=10000000)

X_train_copy = X_train.copy()
j = 0
for language in tqdm(X_train_copy["language"].value_counts().index):
    j += 1
    if j > done and language != "fr":
        texts_translated = []
        df_lang = X_train_copy["text"].loc[X_train["language"] == language].tolist()

        for i in tqdm(range(0, len(df_lang)), desc=language):
            texts_translated.append(translate_texts(translator, df_lang[i], language))
        print(texts_translated[:10])
        X_train_copy["text"].loc[X_train["language"] == language] = texts_translated
        # save
        X_train_copy.to_csv(PATH_PROCESSED + "X_train_translated.csv")

X_train = pd.read_csv(PATH_PROCESSED + "X_train_translated.csv", index_col=0)
X_train["text"].loc[X_train["language"] == "de"]

print("is NaN :")
indexes = X_train[X_train["text"].isna()].index
df_origin.iloc[indexes]

X_test

In [26]:
X_test = pd.read_csv(PATH_PROCESSED + "X_test_detected_language.csv", index_col=0)

# si error durant la traduction -> reprendre à l'index done
done = 0

# google translate API
translator = google_translator(timeout=10000000)

X_test_copy = X_test.copy()
j = 0
for language in tqdm(X_test_copy["language"].value_counts().index):
    j += 1
    if j > done and language != "fr":
        texts_translated = []
        df_lang = X_test_copy["text"].loc[X_test["language"] == language].tolist()

        for i in tqdm(range(0, len(df_lang)), desc=language):
            texts_translated.append(translate_texts(translator, df_lang[i], language))
        print(texts_translated[:10])
        X_test_copy["text"].loc[X_test["language"] == language] = texts_translated
        # save
        X_test_copy.to_csv(PATH_PROCESSED + "X_test_translated.csv")

X_test = pd.read_csv(PATH_PROCESSED + "X_test_translated.csv", index_col=0)
X_test["text"].loc[X_test["language"] == "de"]

print("is NaN :")
indexes = X_test[X_test["text"].isna()].index
df_origin.iloc[indexes]

  0%|          | 0/29 [00:00<?, ?it/s]

en:   0%|          | 0/295 [00:00<?, ?it/s]

["Xbox One à l'édition Call of Duty Advanced Warfare ", 'Kinect Joy Ride Jeu Kinect Jeu Xbox ', 'Annuaire de conception néerlandaise ', 'Enfants Soft Plux Doll Simulation mignon Cat Artisanat Toy Car Decoration GiftZen Enfants Soft Plux Dol Doltesimulation Cat Crafts Toy Car Decoration Cadeau Gifts Les décorations ou les cadeaux de vacances Cat est suffisant pour que vous vous sentiez mieux. ', 'Bouteille de vins Sacs de couverture Décoration Home Party Santa Claus Christmashpp Christmas Gift Sac Sac Candy Merry Christmas Sacs de bonbons de Noël décorféature de haute qualité PC MATÉRIAU Arbres de Noël ou décoration de lieu et ainsi de suite contenu du paquet x sac cadeau de Noël sac de bonbons joyeux Noël sacs de bonbons décor de Noël sans pomme ', 'Les Berenstain Bears visitent le dentiste ', 'Le jeu de chenilles très affamé PA Game de comptage des couleurs et des contrastes P ', "Couleur métallique de luxe Skin étanche PVC Stickers pour DJI Osmo Pocketzen Générique Couleur métallique

/var/folders/_z/rfghqg0j31d_t9y5_yrj2ss00000gn/T/ipykernel_52227/1665405426.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_copy["text"].loc[X_test["language"] == language] = texts_translated


de:   0%|          | 0/113 [00:00<?, ?it/s]

...

sk:   0%|          | 0/1 [00:00<?, ?it/s]

['Robo force ennemi le dictateur ']


/var/folders/_z/rfghqg0j31d_t9y5_yrj2ss00000gn/T/ipykernel_52227/1665405426.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_copy["text"].loc[X_test["language"] == language] = texts_translated


is NaN :


,designation,description,productid,imageid


# Nettoyage et lemmatisation

In [27]:
def clean(text) -> list:
    """
    This function removes stop words from a given text and returns a list of sentences without stop words. It also removes unnecessary spaces, converts all characters to lowercase, and trims leading and trailing spaces.

    Parameters:
        text (str or list): The input text can be either a string containing multiple sentences separated by newline characters ('\n') or a list of strings representing individual sentences.

    Returns:
        list: A list of sentences without stop words and other modifications as described above.
    """

    # Initialize empty list for sentences
    sentences = []
    lemmes = []

    # Pre-compile regex patterns
    SPE_CHAR_RE = re.compile("[^a-zA-ZÀ-ÿ]")
    SPACES_RE = re.compile(r"\s+")

    # Convert 'ortho' column to lowercase and strip spaces, then create a dictionary for quick lookup
    lexique_dict = {
        row["ortho"].lower().strip(): row["lemme"]
        for _, row in lexique_filtred_unique.iterrows()
    }

    # Iterate over each sentence in the text
    for sentence in tqdm(text):
        if sentence is np.nan:
            sentences.append(sentence)
            lemmes.append(sentence)
        else:
            # Replace special characters with spaces while keeping accents
            sentence = SPE_CHAR_RE.sub(" ", sentence)

            # Convert sentence to lowercase and split
            words = nltk.word_tokenize(sentence.lower(), language="french")

            # Look up lemmes in the pre-processed dictionary
            lemme_words = [lexique_dict.get(word, word) for word in words]

            # Join words back into a sentence and strip leading/trailing spaces
            cleaned_sentence = SPACES_RE.sub(" ", " ".join(words)).strip()
            cleaned_lemmes = SPACES_RE.sub(" ", " ".join(lemme_words)).strip()

            # Append the cleaned sentence to the list of sentences
            sentences.append(cleaned_sentence)
            lemmes.append(cleaned_lemmes)

    return sentences, lemmes

X_train

In [26]:
X_train = pd.read_csv(PATH_PROCESSED + "X_train_translated.csv", index_col=0)
# X_train["text"] = strip_accents(X_train["text"])
X_train["text"], X_train["lemmes"] = clean(X_train["text"])
# len texts
X_train["len_text"] = X_train["text"].str.len()
X_train["len_lemmes"] = X_train["lemmes"].str.len()

# add target column
X_train[target.columns[0]] = target

X_train.drop(["designation", "description", "language"], axis="columns", inplace=True)

# save dataframe
X_train.to_csv(PATH_PROCESSED + "X_train_preprocessed.csv")
display(X_train.head())
X_train.describe()

  0%|          | 0/84916 [00:00<?, ?it/s]

,productid,imageid,text,lemmes,len_text,len_lemmes,prdtypecode
0,3804725264,1263597046,pages de carnet personnalisées olivia dot grid...,page de carnet personnalisé olivia dot grid ca...,68,64,10
1,436067568,1008141237,journal des arts le n du l art et son marche s...,journal des art le ne du l art et son marche s...,177,172,2280
2,201115110,938777978,grand stylet ergonomique bleu gamepad nintendo...,grand stylet ergonomique bleu gamepad nintendo...,731,732,50
3,50418756,457047496,peluche donald europe disneyland marionnette à...,peluche donald europe disneyland marionnette à...,52,52,1280
4,278535884,1077757786,la guerre des tuques luc a des idées de grande...,la guerre des tuques luc a des idée de grandeu...,203,212,2705


,productid,imageid,len_text,len_lemmes,prdtypecode
count,8.491600e+04,8.491600e+04,84916.000000,84916.000000,84916.000000
mean,2.555468e+09,1.152691e+09,520.531537,517.319422,1773.219900
std,1.588656e+09,1.751427e+08,664.802838,660.806625,788.179885
min,1.839120e+05,6.728400e+04,4.000000,4.000000,10.000000
25%,6.760519e+08,1.056269e+09,57.000000,56.000000,1281.000000
50%,3.190506e+09,1.213354e+09,265.000000,263.000000,1920.000000
75%,3.995599e+09,1.275646e+09,806.000000,803.000000,2522.000000
max,4.252012e+09,1.328824e+09,11919.000000,12066.000000,2905.000000


X_test

In [28]:
X_test = pd.read_csv(PATH_PROCESSED + "X_test_translated.csv", index_col=0)
# X_test["text"] = strip_accents(X_test["text"])
X_test["text"], X_test["lemmes"] = clean(X_test["text"])
# len texts
X_test["len_text"] = X_test["text"].str.len()
X_test["len_lemmes"] = X_test["lemmes"].str.len()

X_test.drop(["designation", "description", "language"], axis="columns", inplace=True)

# save dataframe
X_test.to_csv(PATH_PROCESSED + "X_test_preprocessed.csv")
display(X_test.head())
X_test.describe()

  0%|          | 0/13812 [00:00<?, ?it/s]

,productid,imageid,text,lemmes,len_text,len_lemmes
84916,516376098,1019294171,folkmanis puppets marionnette et théâtre mini ...,folkmanis puppets marionnette et théâtre mini ...,52,52
84917,133389013,1274228667,porte flamme gaxix flamebringer gaxix u twilig...,porte flamme gaxix flamebringer gaxix u twilig...,63,62
84918,4128438366,1295960357,pompe de filtration speck badu,pompe de filtration speck badu,30,30
84919,3929899732,1265224052,robot de piscine électrique ce robot de piscin...,robot de piscine électrique ce robot de piscin...,937,934
84920,152993898,940543690,hsm destructeur securio c coupe crois e x mm,hsm destructeur securio c coupe croire 2e x mm,44,46


,productid,imageid,len_text,len_lemmes
count,1.381200e+04,1.381200e+04,13812.000000,13812.000000
mean,2.549060e+09,1.153300e+09,520.550463,517.218650
std,1.593114e+09,1.704741e+08,695.102422,690.471314
min,1.847940e+05,4.826610e+05,6.000000,6.000000
25%,6.144059e+08,1.055618e+09,56.000000,56.000000
50%,3.195801e+09,1.212607e+09,253.000000,250.000000
75%,3.993171e+09,1.275573e+09,805.000000,803.000000
max,4.252011e+09,1.328823e+09,19483.000000,19169.000000


# WORDCLOUD mots normaux

In [ ]:
for code in X_train["prdtypecode"].unique():
    # make a map of word of X_train['text']
    word_map = {}
    total_text = ""
    code_df = X_train["text"].loc[X_train["prdtypecode"] == code]
    print(f"########### {code} ###########")
    for i in tqdm(code_df.index, total=len(code_df)):
        text = code_df[i]
        if text is not np.nan:
            total_text += text + " "
            for j in text.split(" "):
                if j not in word_map and j != "":
                    word_map[j] = 1
                elif j != "":
                    word_map[j] += 1

    # sort the map by value
    word_map = sorted(word_map.items(), key=lambda x: x[1], reverse=True)

    # Setting up the subplot for bar plot and word cloud
    fig, axs = plt.subplots(
        1, 2, figsize=FIGSIZE
    )  # Corrected to subplots for creating a 1x2 grid
    fig.suptitle(f"Most frequent words of prdtypecode: {code}")  # Corrected method name

    # Bar plot
    n_words = 30
    axs[0].bar([i[0] for i in word_map[:n_words]], [i[1] for i in word_map[:n_words]])
    axs[0].tick_params(
        axis="x", rotation=90
    )  # Corrected method for setting x-ticks rotation

    # Word cloud
    wordcloud = WordCloud(
        background_color="white",
        max_words=500,
        width=640,
        height=360,
        collocations=False,
    ).generate(total_text)
    axs[1].imshow(wordcloud, interpolation="bilinear")
    axs[1].axis("off")  # Moved inside the loop to apply to each word cloud subplot

    plt.tight_layout(
        rect=[0, 0, 1, 0.96]
    )  # Adjust layout to not overlap with the suptitle
    plt.show()

# WORDCLOUD lemmes

In [ ]:
for code in X_train["prdtypecode"].unique():
    # make a map of word of X_train['text']
    word_map = {}
    total_text = ""
    code_df = X_train["lemmes"].loc[X_train["prdtypecode"] == code]
    print(f"########### {code} ###########")
    for i in tqdm(code_df.index, total=len(code_df)):
        text = code_df[i]
        if text is not np.nan:
            total_text += text + " "
            for j in text.split(" "):
                if j not in word_map and j != "":
                    word_map[j] = 1
                elif j != "":
                    word_map[j] += 1

    # sort the map by value
    word_map = sorted(word_map.items(), key=lambda x: x[1], reverse=True)

    # Setting up the subplot for bar plot and word cloud
    fig, axs = plt.subplots(
        1, 2, figsize=FIGSIZE
    )  # Corrected to subplots for creating a 1x2 grid
    fig.suptitle(
        f"Most frequent lemmes of prdtypecode: {code}"
    )  # Corrected method name

    # Bar plot
    n_words = 30
    axs[0].bar([i[0] for i in word_map[:n_words]], [i[1] for i in word_map[:n_words]])
    axs[0].tick_params(
        axis="x", rotation=90
    )  # Corrected method for setting x-ticks rotation

    # Word cloud
    wordcloud = WordCloud(
        background_color="white",
        max_words=500,
        width=640,
        height=360,
        collocations=False,
    ).generate(total_text)
    axs[1].imshow(wordcloud, interpolation="bilinear")
    axs[1].axis("off")  # Moved inside the loop to apply to each word cloud subplot

    plt.tight_layout(
        rect=[0, 0, 1, 0.96]
    )  # Adjust layout to not overlap with the suptitle
    plt.show()

# WORDCLOUD sans pre-processing

In [ ]:
df_origin[target.columns[0]] = target
# join texts
df_origin["text"] = np.where(
    df_origin["description"].isna(),
    df_origin["designation"].astype(str),
    df_origin["designation"].astype(str) + " " + df_origin["description"].astype(str),
)
for code in df_origin["prdtypecode"].unique():
    # make a map of word of X_train['text']
    word_map = {}
    total_text = ""
    code_df = df_origin["text"].loc[df_origin["prdtypecode"] == code]
    print(f"########### {code} ###########")
    for i in tqdm(code_df.index, total=len(code_df)):
        text = code_df[i]
        total_text += text + " "
        for j in text.split(" "):
            if j not in word_map and j != "":
                word_map[j] = 1
            elif j != "":
                word_map[j] += 1

    # sort the map by value
    word_map = sorted(word_map.items(), key=lambda x: x[1], reverse=True)

    # Setting up the subplot for bar plot and word cloud
    fig, axs = plt.subplots(
        1, 2, figsize=FIGSIZE
    )  # Corrected to subplots for creating a 1x2 grid
    fig.suptitle(
        f"Most frequent words (without cleaning) of prdtypecode: {code}"
    )  # Corrected method name

    # Bar plot
    n_words = 30
    axs[0].bar([i[0] for i in word_map[:n_words]], [i[1] for i in word_map[:n_words]])
    axs[0].tick_params(
        axis="x", rotation=90
    )  # Corrected method for setting x-ticks rotation

    # Word cloud
    wordcloud = WordCloud(
        background_color="white",
        max_words=500,
        width=640,
        height=360,
        collocations=False,
    ).generate(total_text)
    axs[1].imshow(wordcloud, interpolation="bilinear")
    axs[1].axis("off")  # Moved inside the loop to apply to each word cloud subplot

    plt.tight_layout(
        rect=[0, 0, 1, 0.96]
    )  # Adjust layout to not overlap with the suptitle
    plt.show()